In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

#mae_list1 = []
#mape_list1 = []
#for i in range(20):
#    print(i)
airports = ['VABB', 'VOTV', 'VOBL', 'VECC', 'VIDP']

def get_visibility(code):
    arr = np.load("visibility_arr.npy")
    idx = airports.index(code)
    return arr[:, idx].astype(np.float32)

def get_era_full(param, level):
    arr = np.load("18To20{}{}_uint8.npy".format(param, level))
    return (arr/256)#.astype(np.float32)

# Import data
params = ["z", "z", "z"]
levels = [500, 700, 1000]

in1_var = get_era_full(params[0], levels[0])
in2_var = get_era_full(params[1], levels[1])
in3_var = get_era_full(params[2], levels[2])

# data for a given airport
X = np.concatenate((np.expand_dims(in1_var, axis=3), np.expand_dims(in2_var, axis=3), np.expand_dims(in3_var, axis=3)), axis=3)
Y = get_visibility('VABB').reshape(-1, 1)

import sklearn
from sklearn.model_selection import train_test_split

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, Y_train, Y_rem = train_test_split(X,Y, train_size=0.7)

# Now since we want the valid and test size to be equal. 
X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

#to clear space for gpu, if occupied by any process
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

# model

initializer = tf.keras.initializers.HeUniform()
inputs = keras.Input(shape=(157, 157, 3))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu", kernel_initializer = initializer)(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu", kernel_initializer = initializer)(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu", kernel_initializer = initializer)(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu", kernel_initializer = initializer)(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu", kernel_initializer = initializer)(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
top_dropout_rate = 0.4
x = layers.Dropout(top_dropout_rate)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs, name="Basic2dCNN")

# compiling the model
opt =tf.keras.optimizers.RMSprop(learning_rate=0.002)
model.compile(loss = 'mse', optimizer = opt, metrics = [tf.keras.losses.MeanAbsoluteError()])
callbacks = [keras.callbacks.ModelCheckpoint(filepath = f"vVABB_2D.keras",
                                            save_best_only = True, monitor = "val_loss")]

history = model.fit(X_train, Y_train, batch_size = 64, epochs = 100, validation_data = (X_valid, Y_valid), verbose = 0, callbacks = callbacks)

import matplotlib.pyplot as plt
%matplotlib inline
f1 = plt.figure();
mae = history.history["mean_absolute_error"]
loss = history.history["loss"]
mape = history.history["mean_absolute_percentage_error"]
val_mae = history.history["val_mean_absolute_error"]
val_loss = history.history["val_loss"]
val_mape = history.history["val_mean_absolute_percentage_error"]
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, "bo", label = "Training loss(MSE)")
plt.plot(epochs, val_loss, "b", label = "Validation loss(MSE)")
plt.title("Training and validation loss(MSE)")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()

f2 = plt.figure();
plt.plot(epochs, mae, "bo", label = "Training accuracy(MAE)")
plt.plot(epochs, val_mae, "b", label = "Validation accuracy(MAE)")
plt.title("Training and validation accuracy(MAE)")
plt.xlabel("epochs")
plt.ylabel("MAE")
plt.legend()
plt.show()

test_loss, test_mae = model.evaluate(X_test, Y_test)
print(f"Test MAE: {test_mae:.3f}")
#mae_list1.append(test_mae)
#mape_list1.append(test_mape)
f1.savefig(f"vVABBl_2D.jpg", bbox_inches='tight', dpi=600);
f2.savefig(f"vVABBm_2D.jpg", bbox_inches='tight', dpi=600);
#f3.savefig(f"vVABB{i}mape_2D.jpg", bbox_inches='tight', dpi=600);
    
#print(mae_list1)
#print(mape_list1)

In [ ]:
import numpy as np
mae_list1 = [0.4781510531902313, 0.4756133258342743, 0.4818015694618225, 0.47948676347732544, 0.47266677021980286, 0.4710808992385864, 0.46902909874916077, 0.47727319598197937, 0.4791882038116455, 0.489938884973526, 0.4724079370498657, 0.46932166814804077, 0.4712788462638855, 0.4628233313560486, 0.4710977375507355, 0.4838845729827881, 0.48105090856552124, 0.46335792541503906, 0.4775252044200897, 0.48078182339668274]
a = np.array(mae_list1)
np.mean(a), np.std(a), a

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

mae_list2 = []
for i in range(20):
    print(i)
    airports = ['VABB', 'VOTV', 'VOBL', 'VECC', 'VIDP']

    def get_visibility(code):
        arr = np.load("visibility_arr.npy")
        idx = airports.index(code)
        return arr[:, idx].astype(np.float32)

    def get_era_full(param, level):
        arr = np.load("18To20{}{}_uint8.npy".format(param, level))
        return arr.astype(np.float32)

    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    # data for a given airport
    X = np.concatenate((np.expand_dims(in1_var, axis=3), np.expand_dims(in2_var, axis=3), np.expand_dims(in3_var, axis=3)), axis=3)
    Y = get_visibility('VOTV').reshape(-1, 1)

    import sklearn
    from sklearn.model_selection import train_test_split

    # In the first step we will split the data in training and remaining dataset
    X_train, X_rem, Y_train, Y_rem = train_test_split(X,Y, train_size=0.7)

    # Now since we want the valid and test size to be equal. 
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

    #to clear space for gpu, if occupied by any process
    import tensorflow as tf
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.compat.v1.Session(config=config)

    conv_base = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(157, 157, 3))

    # preprocess_input

    train_features = tf.keras.applications.vgg16.preprocess_input(X_train)
    val_features = tf.keras.applications.vgg16.preprocess_input(X_valid)
    test_features = tf.keras.applications.vgg16.preprocess_input(X_test)

    train_features = conv_base.predict(X_train)
    val_features = conv_base.predict(X_valid)
    test_features = conv_base.predict(X_test)

    train_labels = Y_train
    val_labels = Y_valid
    test_labels = Y_test

    # model building
    inputs = keras.Input(shape = (4,4,512))
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs, name="vgg16")

    # compiling the model
    opt =tf.keras.optimizers.RMSprop(learning_rate=0.002)
    model.compile(loss = 'mse', optimizer = opt, metrics = [tf.keras.losses.MeanAbsoluteError()])
    callbacks = [keras.callbacks.ModelCheckpoint(filepath = f"vVOTV {i}_vgg16.keras",
                                                save_best_only = True, monitor = "val_loss")]

    history = model.fit(train_features, train_labels, batch_size = 128, epochs = 100, validation_data = (val_features, val_labels), verbose = 0, callbacks = callbacks)

    import matplotlib.pyplot as plt
    %matplotlib inline
    f1 = plt.figure();
    mae = history.history["mean_absolute_error"]
    loss = history.history["loss"]
    val_mae = history.history["val_mean_absolute_error"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, "bo", label = "Training loss(MSE)");
    plt.plot(epochs, val_loss, "b", label = "Validation loss(MSE)");
    plt.title("Training and validation loss(MSE)");
    plt.legend();

    f2 = plt.figure();
    plt.plot(epochs, mae, "bo", label = "Training accuracy(MAE)");
    plt.plot(epochs, val_mae, "b", label = "Validation accuracy(MAE)");
    plt.title("Training and validation accuracy(MAE)");
    plt.legend();
    plt.show();

    test_loss, test_mae = model.evaluate(test_features, test_labels)
    print(f"Test MAE: {test_mae:.3f}")
    mae_list2.append(test_mae)
    f1.savefig(f"vVOTV{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    f2.savefig(f"vVOTV{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    
print(mae_list2)

In [ ]:
b = np.array(mae_list2)
np.mean(b), np.std(b), b

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

mae_list3 = []
for i in range(20):
    print(i)
    airports = ['VABB', 'VOTV', 'VOBL', 'VECC', 'VIDP']

    def get_visibility(code):
        arr = np.load("visibility_arr.npy")
        idx = airports.index(code)
        return arr[:, idx].astype(np.float32)

    def get_era_full(param, level):
        arr = np.load("18To20{}{}_uint8.npy".format(param, level))
        return arr.astype(np.float32)

    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    # data for a given airport
    X = np.concatenate((np.expand_dims(in1_var, axis=3), np.expand_dims(in2_var, axis=3), np.expand_dims(in3_var, axis=3)), axis=3)
    Y = get_visibility('VOBL').reshape(-1, 1)

    import sklearn
    from sklearn.model_selection import train_test_split

    # In the first step we will split the data in training and remaining dataset
    X_train, X_rem, Y_train, Y_rem = train_test_split(X,Y, train_size=0.7)

    # Now since we want the valid and test size to be equal. 
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

    #to clear space for gpu, if occupied by any process
    import tensorflow as tf
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.compat.v1.Session(config=config)

    conv_base = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(157, 157, 3))

    # preprocess_input

    train_features = tf.keras.applications.vgg16.preprocess_input(X_train)
    val_features = tf.keras.applications.vgg16.preprocess_input(X_valid)
    test_features = tf.keras.applications.vgg16.preprocess_input(X_test)

    train_features = conv_base.predict(X_train)
    val_features = conv_base.predict(X_valid)
    test_features = conv_base.predict(X_test)

    train_labels = Y_train
    val_labels = Y_valid
    test_labels = Y_test

    # model building
    inputs = keras.Input(shape = (4,4,512))
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs, name="vgg16")

    # compiling the model
    opt =tf.keras.optimizers.RMSprop(learning_rate=0.002)
    model.compile(loss = 'mse', optimizer = opt, metrics = [tf.keras.losses.MeanAbsoluteError()])
    callbacks = [keras.callbacks.ModelCheckpoint(filepath = f"vVABB{i}_vgg16.keras",
                                                save_best_only = True, monitor = "val_loss")]

    history = model.fit(train_features, train_labels, batch_size = 128, epochs = 100, validation_data = (val_features, val_labels), verbose = 0, callbacks = callbacks)

    import matplotlib.pyplot as plt
    %matplotlib inline
    f1 = plt.figure();
    mae = history.history["mean_absolute_error"]
    loss = history.history["loss"]
    val_mae = history.history["val_mean_absolute_error"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, "bo", label = "Training loss(MSE)");
    plt.plot(epochs, val_loss, "b", label = "Validation loss(MSE)");
    plt.title("Training and validation loss(MSE)");
    plt.legend();

    f2 = plt.figure();
    plt.plot(epochs, mae, "bo", label = "Training accuracy(MAE)");
    plt.plot(epochs, val_mae, "b", label = "Validation accuracy(MAE)");
    plt.title("Training and validation accuracy(MAE)");
    plt.legend();
    plt.show();

    test_loss, test_mae = model.evaluate(test_features, test_labels)
    print(f"Test MAE: {test_mae:.3f}")
    mae_list3.append(test_mae)
    f1.savefig(f"vVOBL{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    f2.savefig(f"vVOBL{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    
print(mae_list3)

In [ ]:
c = np.array(mae_list3)
np.mean(c), np.std(c), c

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

mae_list4 = []
for i in range(20):
    print(i)
    airports = ['VABB', 'VOTV', 'VOBL', 'VECC', 'VIDP']

    def get_visibility(code):
        arr = np.load("visibility_arr.npy")
        idx = airports.index(code)
        return arr[:, idx].astype(np.float32)

    def get_era_full(param, level):
        arr = np.load("18To20{}{}_uint8.npy".format(param, level))
        return arr.astype(np.float32)

    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    # data for a given airport
    X = np.concatenate((np.expand_dims(in1_var, axis=3), np.expand_dims(in2_var, axis=3), np.expand_dims(in3_var, axis=3)), axis=3)
    Y = get_visibility('VECC').reshape(-1, 1)

    import sklearn
    from sklearn.model_selection import train_test_split

    # In the first step we will split the data in training and remaining dataset
    X_train, X_rem, Y_train, Y_rem = train_test_split(X,Y, train_size=0.7)

    # Now since we want the valid and test size to be equal. 
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

    #to clear space for gpu, if occupied by any process
    import tensorflow as tf
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.compat.v1.Session(config=config)

    conv_base = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(157, 157, 3))

    # preprocess_input

    train_features = tf.keras.applications.vgg16.preprocess_input(X_train)
    val_features = tf.keras.applications.vgg16.preprocess_input(X_valid)
    test_features = tf.keras.applications.vgg16.preprocess_input(X_test)

    train_features = conv_base.predict(X_train)
    val_features = conv_base.predict(X_valid)
    test_features = conv_base.predict(X_test)

    train_labels = Y_train
    val_labels = Y_valid
    test_labels = Y_test

    # model building
    inputs = keras.Input(shape = (4,4,512))
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs, name="vgg16")

    # compiling the model
    opt =tf.keras.optimizers.RMSprop(learning_rate=0.002)
    model.compile(loss = 'mse', optimizer = opt, metrics = [tf.keras.losses.MeanAbsoluteError()])
    callbacks = [keras.callbacks.ModelCheckpoint(filepath = f"vVECC{i}_vgg16.keras",
                                                save_best_only = True, monitor = "val_loss")]

    history = model.fit(train_features, train_labels, batch_size = 128, epochs = 100, validation_data = (val_features, val_labels), verbose = 0, callbacks = callbacks)

    import matplotlib.pyplot as plt
    %matplotlib inline
    f1 = plt.figure();
    mae = history.history["mean_absolute_error"]
    loss = history.history["loss"]
    val_mae = history.history["val_mean_absolute_error"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, "bo", label = "Training loss(MSE)");
    plt.plot(epochs, val_loss, "b", label = "Validation loss(MSE)");
    plt.title("Training and validation loss(MSE)");
    plt.legend();

    f2 = plt.figure()
    plt.plot(epochs, mae, "bo", label = "Training accuracy(MAE)");
    plt.plot(epochs, val_mae, "b", label = "Validation accuracy(MAE)");
    plt.title("Training and validation accuracy(MAE)");
    plt.legend();
    plt.show();

    test_loss, test_mae = model.evaluate(test_features, test_labels)
    print(f"Test MAE: {test_mae:.3f}")
    mae_list4.append(test_mae)
    f1.savefig(f"vVECC{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    f2.savefig(f"vVECC{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    
print(mae_list4)

In [ ]:
d = np.array(mae_list4)
np.mean(d), np.std(d), d

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

mae_list5 = []
for i in range(20):
    print(i)
    airports = ['VABB', 'VOTV', 'VOBL', 'VECC', 'VIDP']

    def get_visibility(code):
        arr = np.load("visibility_arr.npy")
        idx = airports.index(code)
        return arr[:, idx].astype(np.float32)

    def get_era_full(param, level):
        arr = np.load("18To20{}{}_uint8.npy".format(param, level))
        return arr.astype(np.float32)

    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    # data for a given airport
    X = np.concatenate((np.expand_dims(in1_var, axis=3), np.expand_dims(in2_var, axis=3), np.expand_dims(in3_var, axis=3)), axis=3)
    Y = get_visibility('VIDP').reshape(-1, 1)

    import sklearn
    from sklearn.model_selection import train_test_split

    # In the first step we will split the data in training and remaining dataset
    X_train, X_rem, Y_train, Y_rem = train_test_split(X,Y, train_size=0.7)

    # Now since we want the valid and test size to be equal. 
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

    #to clear space for gpu, if occupied by any process
    import tensorflow as tf
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.compat.v1.Session(config=config)

    conv_base = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(157, 157, 3))

    # preprocess_input

    train_features = tf.keras.applications.vgg16.preprocess_input(X_train)
    val_features = tf.keras.applications.vgg16.preprocess_input(X_valid)
    test_features = tf.keras.applications.vgg16.preprocess_input(X_test)

    train_features = conv_base.predict(X_train)
    val_features = conv_base.predict(X_valid)
    test_features = conv_base.predict(X_test)

    train_labels = Y_train
    val_labels = Y_valid
    test_labels = Y_test

    # model building
    inputs = keras.Input(shape = (4,4,512))
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs, name="vgg16")

    # compiling the model
    opt =tf.keras.optimizers.RMSprop(learning_rate=0.002)
    model.compile(loss = 'mse', optimizer = opt, metrics = [tf.keras.losses.MeanAbsoluteError()])
    callbacks = [keras.callbacks.ModelCheckpoint(filepath = f"vVIDP{i}_vgg16.keras",
                                                save_best_only = True, monitor = "val_loss")]

    history = model.fit(train_features, train_labels, batch_size = 128, epochs = 100, validation_data = (val_features, val_labels), verbose = 0, callbacks = callbacks)

    import matplotlib.pyplot as plt
    %matplotlib inline
    f1 = plt.figure();
    mae = history.history["mean_absolute_error"]
    loss = history.history["loss"]
    val_mae = history.history["val_mean_absolute_error"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, "bo", label = "Training loss(MSE)");
    plt.plot(epochs, val_loss, "b", label = "Validation loss(MSE)");
    plt.title("Training and validation loss(MSE)");
    plt.legend();

    f2 = plt.figure();
    plt.plot(epochs, mae, "bo", label = "Training accuracy(MAE)");
    plt.plot(epochs, val_mae, "b", label = "Validation accuracy(MAE)");
    plt.title("Training and validation accuracy(MAE)");
    plt.legend();
    plt.show();

    test_loss, test_mae = model.evaluate(test_features, test_labels)
    print(f"Test MAE: {test_mae:.3f}")
    mae_list5.append(test_mae)
    f1.savefig(f"vVIDP{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    f2.savefig(f"vVIDP{i}_vgg16.jpg", bbox_inches='tight', dpi=600);
    
print(mae_list5)

In [ ]:
e = np.array(mae_list5)
np.mean(e), np.std(e), e

In [ ]:
import pandas as pd
x = pd.Series(mae_list5)
x.plot.hist(bins=50, figsize=[10, 5])
plt.xlabel("visibility");